In [1]:
import re
import numpy as np
import pandas as pd
import os
import json

In [2]:
# Importing libraries which will assist in removing the html tags
from io import StringIO
from html.parser import HTMLParser

"""Remove html tags from messages"""

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [3]:
directory = 'txt_files/Raw_v2'
faq_dict = {}
counter = 0

for filename in os.listdir(directory):
    if (filename.endswith(".txt") and 'blog_' not in filename):
        f = open('txt_files/Raw_v2/{}'.format(filename), 'r+', encoding='utf-8')
        string_list = f.readlines()
        
        if len(string_list) > 0:
            
            # Cleans blank lines and random sub-headings from the text (Part 1 of 2)
            new_lines = [line for line in string_list if line != '\n' and line != ' \n']
            new_lines = [strip_tags(line) for line in new_lines]
            new_lines = [re.sub(' +', ' ', line) for line in new_lines]
            new_lines = [re.sub('\xa0', '', line) for line in new_lines]
            for i in range(len(new_lines) - 1):
                if new_lines[i] == new_lines[i+1]:
                    new_lines[i] = ' '
            for i in range(len(new_lines)):
                if new_lines[i].startswith(' '):
                    new_lines[i] = ' '
            new_lines = [line for line in new_lines if line != ' ']

            try:
                # Look for questions that aren't in headings
                for i in range(len(new_lines)):
                    if '?' in new_lines[i]:
                        if new_lines[i] in faq_dict.keys():
                            a = faq_dict[new_lines[i]]
                            if new_lines[i + 1] not in a:
                                a.append(new_lines[i + 1])
                                faq_dict[new_lines[i]] = a
                        else:
                            faq_dict[new_lines[i]] = [new_lines[i + 1]]

                # Look for questions in H2 headings
                    if ('?' in new_lines[i] and 'H2' in new_lines[i]):
                        key = new_lines[i]
                        vals = []
                        j = i + 1
                        while 'H2' not in new_lines[j]:
                            vals.append(new_lines[j])
                            j += 1
                        if key in faq_dict.keys():
                            a = faq_dict[key]
                            for k in range(len(vals)):
                                if vals[k] not in a:
                                    a.append(vals[k])
                                    faq_dict[key] = a
                        else:
                            faq_dict[key] = vals

                # Look for questions in H3 headings
                    if ('?' in new_lines[i] and 'H3' in new_lines[i]):
                        key = new_lines[i]
                        vals = []
                        j = i + 1
                        while 'H2' not in new_lines[j]:
                            vals.append(new_lines[j])
                            j += 1
                        if key in faq_dict.keys():
                            a = faq_dict[key]
                            for k in range(len(vals)):
                                if vals[k] not in a:
                                    a.append(vals[k])
                                    faq_dict[key] = a
                        else:
                            faq_dict[key] = vals
            except:
                print('Skipped file: {}'.format(filename))
            
            # Cleans blank lines and random sub-headings from the text (Part 2 of 2)
            new_lines = [line for line in new_lines if line not in faq_dict.keys() and line not in faq_dict.values() and 'FAQ' not in line]
            new_lines = [re.sub('H\d: ', '', line) for line in new_lines]
            new_lines = [re.sub('Paragraph: ', '', line) for line in new_lines]
            new_lines = [line for line in new_lines if line != '\n' and line != ' \n']
            final_lines = []
            final_lines = [line for line in new_lines if line not in final_lines]
            final_lines = list(dict.fromkeys(final_lines))

            # Write cleaned text data to new text file
            if len(final_lines) > 0:
                f_cl = open('txt_files/Cleaned_v2/cleaned_{}'.format(filename), 'w+', encoding='utf-8')
                for i in range(len(final_lines)):
                    f_cl.write(final_lines[i])
                f_cl.close()
        f.close()
    counter += 1
    print('Complete {0} {1}'.format(counter, filename))

Complete 1 explore_.txt
Complete 2 explore_about-us_our-courses.txt
Complete 3 explore_about-us_our-values.txt
Complete 4 explore_alumni_hire-explorer.txt
Complete 5 explore_alumni_student-reviews.txt
Complete 6 explore_contact.txt
Complete 7 explore_course_info_1_3.txt
Complete 8 explore_course_info_1_3_1.txt
Complete 9 explore_course_info_1_3_oncampus.txt
Complete 10 explore_course_info_1_3_online.txt
Complete 11 explore_course_info_2_4.txt
Complete 12 explore_course_info_34_3.txt
Complete 13 explore_course_info_34_3_1.txt
Complete 14 explore_course_info_34_3_online.txt
Complete 15 explore_course_info_35_4.txt
Complete 16 explore_course_info_38_5.txt
Complete 17 explore_course_info_38_5_online.txt
Complete 18 explore_course_info_39_5.txt
Complete 19 explore_course_info_39_5_online.txt
Complete 20 explore_course_info_3_3.txt
Complete 21 explore_course_info_3_3_1.txt
Complete 22 explore_course_info_3_3_online.txt
Complete 23 explore_course_info_40_5.txt
Complete 24 explore_course_info_

In [4]:
final_lines

['https://explore-datascience.net/signup\n',
 'Welcome To Our Universe!\n',
 'CREATE MY ACCOUNT\n',
 'Ok\n',
 'ok\n']

In [5]:
for key in faq_dict.keys():
    print('\n')
    print(key)
    print(faq_dict[key])



Talented, eager and hampered by access to funding? Apply for a loan to finance your education. EXPLORE aims to provide affordable rates for students who meet our criteria.

['Why EXPLORE?\n', 'Our Payment Plans\n', 'Upfront Payment\n', 'EXPLORE Student Loan\n']


Why EXPLORE?

['We pride ourselves on our innovative, practical and real-world-readiness approach to teaching and learning. Our scientists, who facilitate all our programmes, have experience solving difficult problems across a variety of industries in South Africa and abroad.\n']


EXPLORE is Africa’s largest Data Science Academy - thousands of students have trusted us with their futures and not been disappointed - what are you waiting for? 

['Transform your workforce.\n', 'Why EXPLORE?\n']


Why Us?

['Enterprise\n']


What are you waiting for? 

['I came from a business focused role. I had a desire to break into tech, more specifically Data Science and EXPLORE’s Data Science program became the catalyst for that ambition t

In [6]:
# Manual checks
# Deleted files:
### 1. faq#Pricingcollapse3.txt and cleaned_faq#Pricingcollapse3.txt: No useful info
### 2. faq.txt and cleaned_faq.txt: No useful info

# Other info:
### 1. Many of the text files have info that also appears in the faq_dict. This info was not deleted because
###    it was recommended to leave as much informative text data in the files as possible.
### 2. contact.txt required email addresses. These were manually added.
### 3. Removed duplicated text from individual files. Some info is duplicated across files, this was not removed.
### 4. cleaned_data-engineering.txt: email address required on line 118, could not find it on website.





## Joining QA with annotated data

In [7]:
with open('./answers.json', encoding='utf-8') as f:
    data = json.load(f)
    f.close()
questions = data['data'][0]['paragraphs'][0]['qas']
ids = []

for question in questions:
    ids.append(question['id'])
max_id = max(ids)
extracted_faqs = []
for key in range(len(faq_dict.keys())):
    new_questions = {'question':str(),
                'id': int(),
                'answers':[{'text':str(),
                          'answer_start': int(), # Problem: need to be able to get start token from files
                          'answer_category': None}],
                'is_impossible': False}
    new_questions['question'] = list(faq_dict.keys())[key]
    # Cleaning question
    new_questions['question'] = re.sub("H1: ", "", new_questions['question'])
    new_questions['question'] = re.sub("H2: ", "", new_questions['question'])
    new_questions['question'] = re.sub("H3: ", "", new_questions['question'])
    new_questions['question'] = re.sub("\n", "", new_questions['question'])

    new_questions['id'] = max_id + key
    new_questions['answers'][0]['text'] = " ".join(faq_dict[list(faq_dict.keys())[key]])

    # cleaning answers
    new_questions['answers'][0]['text'] = re.sub("\n", "", new_questions['answers'][0]['text'])
    new_questions['answers'][0]['text'] = re.sub("H1", "", new_questions['answers'][0]['text'])
    new_questions['answers'][0]['text'] = re.sub("H2", "", new_questions['answers'][0]['text'])
    new_questions['answers'][0]['text'] = re.sub(r"H3: ", "", new_questions['answers'][0]['text'])
    new_questions['answers'][0]['text'] = re.sub("Paragraph: ", "", new_questions['answers'][0]['text'])
    new_questions['answers'][0]['text'] = re.sub("’", "'", new_questions['answers'][0]['text'])

    new_questions['answers'][0]['answer_start'] = None
    new_questions['answers'][0]['answer_category'] = None
    new_questions['is_impossible'] = False
    extracted_faqs.append(new_questions)

appended_questions =  questions + extracted_faqs
data['data'][0]['paragraphs'][0]['qas'] = appended_questions
with open('./training_data.json', 'w', encoding = 'utf-8') as f:
    json.dump(data, f)
    f.close()

In [8]:
import csv
csv_columns = ['Answer']
dict_data = [
{'No': 1, 'Name': 'Alex', 'Country': 'India'},
{'No': 2, 'Name': 'Ben', 'Country': 'USA'},
{'No': 3, 'Name': 'Shri Ram', 'Country': 'India'},
{'No': 4, 'Name': 'Smith', 'Country': 'USA'},
{'No': 5, 'Name': 'Yuva Raj', 'Country': 'India'},
]
csv_file = "Names.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error")

ValueError: dict contains fields not in fieldnames: 'Country', 'No', 'Name'